# Desafio 3 - Modelagem

A PagShow é uma casa de eventos muito famosa em São Paulo. 

Os clientes compram os ingressos pelo site da PagShow e recebem um QR Code que será usado em catracas para liberar o acesso.

Quando os clientes passam pelas catracas, são capturados dados dos clientes e eventos conforme a tabela ACESSOS_CATRACA.

(Schema e Sample abaixo).

**Schema**

`ACESSOS_CATRACA`
| Name | Type | Description |
|------|------|-------------|
| evento_nome | Varchar | Nome do Evento |
| evento_inicio | Timestamp | Data de Início do Evento |
| artista_nome | Varchar | Nome do Artista do Evento |
| cliente_nome | Varchar | Nome do Cliente |
| cliente_cpf | Varchar | CPF do Cliente |
| cliente_telefone | Varchar | Telefone do Cliente |
| cliente_entrada | Timestamp | Data e Hora que o cliente chegou ao evento |

**Sample**

| evento nome | evento Inlclo | artista nome | cliente nome | cliente | ciente telefone | ciente entrada |
|-|-|-|-|-|-|-|
| Funn Festival | 01/04/2023 22:00 | Bruno & Mamone | Lucas | 111.111.111-11 | 5513992123212 | 01/04/2023 22:43 | 
| Fuzuê | 02/04/2023 22:00 | Michel Teló | Pedro | 222.222.222-22 | 5511982332556 | 02/04/2023 22:31 |
| Comédia Stand Up | 03/04/2023 22:00 | Fábio Rabin | Leonardo | 333.333.333-33 | 5513991857732 | 03/04/2023 22:12 |
| Fuzuê|  04/04/2023 22:00 | Michel Teló | Vivian | 444.444.444-44 | 5521997889889 | 04/04/2023 22:01 |
| 2 Minutes To Midnight | 05/04/2023 22:00 | Iron Maiden | Anne | 555.555.555-55 | 5511988272121 | 05/04/2023 22:21 |
| Comédia Stand Up | 03/04/2023 22:00 | Fábio Rabin | Lucas | 111.111.111-11 | 5513992123212 | 03/04/2023 22:04 |
| 2 Minutes To Midnight | 07/04/2023 22:00 | Iron Maiden | Rodrigo | 666.666.666-66 | 5511945311288 | 07/04/2023 22:06 |
| Funn Festival | 01/04/2023 22:00 | Bruno & Marrone | Pedro | 222.222.222-22 | 5511982332556 | 01/04/2023 22:40 |
| Oceano | 09/04/2023 22:00 | Djavan | Suellen | 777.777.777-77 | 5521991012434 | 09/04/2023 23:06 |


1. Faça graficamente uma modelagem dimensional considerando a tabela acima e seus
respectivos dados.
    1. Obs: Caso precise inferir alguma informação, deixe claro quais foram os
pontos inferidos.
2. Após a modelagem finalizada, elabore consultas SQL, que responda as perguntas
abaixo utilizando o modelo criado:
    1. Quantas pessoas chegaram após o horário de início de cada evento?
    2. Quantos eventos aconteceram no dia de sábado?

## Modelagem

![Diagrama](star_schema.png "Modelagem em diagrama")

### Tabelas e Campos:

`Dim_Evento`

Tabela que armazena informações sobre os eventos.

**Campos:**
- evento_nome: Nome do evento (chave primária).
- evento_inicio: Data e hora de início do evento (armazenada no formato ISO 8601).
- artista_nome: Nome do artista que se apresenta no evento.

`Dim_Cliente`

Tabela que contém informações dos clientes.

**Campos:**
- cliente_cpf: CPF do cliente (chave primária).
- cliente_nome: Nome do cliente.
- cliente_telefone: Número de telefone do cliente.

`Dim_Data_Hora`

Tabela auxiliar para armazenar detalhes de data e hora relacionados aos eventos.

**Campos:**
- cliente_entrada: Data e hora de entrada do cliente no evento (chave primária).
- data: Parte da data (dia, mês e ano) da cliente_entrada.
- hora: Parte da hora (hora e minuto) da cliente_entrada.

`Fato_Acessos_Catraca`

Tabela de fatos que registra os acessos dos clientes aos eventos através das catracas.

**Campos:**
- evento_nome: Nome do evento (chave estrangeira referenciando Dim_Evento).
- cliente_cpf: CPF do cliente (chave estrangeira referenciando Dim_Cliente).
- cliente_entrada: Data e hora de entrada do cliente no evento (chave estrangeira referenciando Dim_Data_Hora).


**Onde:**
- Os campos evento_nome, evento_inicio, artista_nome, cliente_nome, cliente_cpf, cliente_telefone e cliente_entrada foram todos diretamente fornecidos na tabela sample.
- As tabelas Dim_Evento, Dim_Cliente, Dim_Data_Hora e Fato_Acessos_Catraca foram estruturadas com base nas informações diretamente disponíveis na tabela sample.

> SQLite does not have a storage class set aside for storing dates and/or times. Instead, the built-in Date And Time Functions of SQLite are capable of storing dates and times as TEXT, REAL, or INTEGER values: TEXT as ISO8601 strings ("YYYY-MM-DD HH:MM:SS.SSS"). [Referência](https://www.sqlite.org/datatype3.html).

## Consultas SQL

### Inicialização do banco de dados

In [1]:
import sqlite3
import pandas as pd
from datetime import datetime

def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except sqlite3.Error as e:
        print(e)
    return conn

db_name = "pagshow.db"
conn = create_connection(db_name)
cursor = conn.cursor()

### Criação das tabelas

In [2]:
table_names = ["Dim_Evento", "Dim_Cliente", "Dim_Data_Hora", "Fato_Acessos_Catraca"]

for table_name in table_names:
    try:
        cursor.execute(f"DROP TABLE IF EXISTS {table_name}")
    except sqlite3.OperationalError as e:
        print(f"Error dropping table '{table_name}': {e}")

In [3]:
create_tables = [
    """
    CREATE TABLE IF NOT EXISTS Dim_Evento (
        evento_nome VARCHAR PRIMARY KEY,
        evento_inicio TEXT,
        artista_nome VARCHAR
    )
    """,
    """
    CREATE TABLE IF NOT EXISTS Dim_Cliente (
        cliente_cpf VARCHAR PRIMARY KEY,
        cliente_nome VARCHAR,
        cliente_telefone VARCHAR
    )
    """,
    """
    CREATE TABLE IF NOT EXISTS Dim_Data_Hora (
        cliente_entrada TEXT PRIMARY KEY,
        data TEXT,
        hora TEXT
    )
    """,
    """
    CREATE TABLE IF NOT EXISTS Fato_Acessos_Catraca (
        evento_nome VARCHAR,
        cliente_cpf VARCHAR,
        cliente_entrada TEXT,
        FOREIGN KEY (evento_nome) REFERENCES Dim_Evento(evento_nome),
        FOREIGN KEY (cliente_cpf) REFERENCES Dim_Cliente(cliente_cpf),
        FOREIGN KEY (cliente_entrada) REFERENCES Dim_Data_Hora(cliente_entrada)
    )
    """
]

for table in create_tables:
    cursor.execute(table)

In [4]:
for table_name in table_names:
    print(f"Table: {table_name}")
    cursor.execute(f"PRAGMA table_info('{table_name}')")
    for row in cursor.fetchall():
        print(row)
    print("\n")

Table: Dim_Evento
(0, 'evento_nome', 'VARCHAR', 0, None, 1)
(1, 'evento_inicio', 'TEXT', 0, None, 0)
(2, 'artista_nome', 'VARCHAR', 0, None, 0)


Table: Dim_Cliente
(0, 'cliente_cpf', 'VARCHAR', 0, None, 1)
(1, 'cliente_nome', 'VARCHAR', 0, None, 0)
(2, 'cliente_telefone', 'VARCHAR', 0, None, 0)


Table: Dim_Data_Hora
(0, 'cliente_entrada', 'TEXT', 0, None, 1)
(1, 'data', 'TEXT', 0, None, 0)
(2, 'hora', 'TEXT', 0, None, 0)


Table: Fato_Acessos_Catraca
(0, 'evento_nome', 'VARCHAR', 0, None, 0)
(1, 'cliente_cpf', 'VARCHAR', 0, None, 0)
(2, 'cliente_entrada', 'TEXT', 0, None, 0)




### Inserção de dados

In [5]:
data = [
    ("Funn Festival", "01/04/2023 22:00", "Bruno & Mamone", "Lucas", "111.111.111-11", "5513992123212", "01/04/2023 22:43"),
    ("Fuzuê", "02/04/2023 22:00", "Michel Teló", "Pedro", "222.222.222-22", "5511982332556", "02/04/2023 22:31"),
    ("Comédia Stand Up", "03/04/2023 22:00", "Fábio Rabin", "Leonardo", "333.333.333-33", "5513991857732", "03/04/2023 22:12"),
    ("Fuzuê", "04/04/2023 22:00", "Michel Teló", "Vivian", "444.444.444-44", "5521997889889", "04/04/2023 22:01"),
    ("2 Minutes To Midnight", "05/04/2023 22:00", "Iron Maiden", "Anne", "555.555.555-55", "5511988272121", "05/04/2023 22:21"),
    ("Comédia Stand Up", "03/04/2023 22:00", "Fábio Rabin", "Lucas", "111.111.111-11", "5513992123212", "03/04/2023 22:04"),
    ("2 Minutes To Midnight", "07/04/2023 22:00", "Iron Maiden", "Rodrigo", "666.666.666-66", "5511945311288", "07/04/2023 22:06"),
    ("Funn Festival", "01/04/2023 22:00", "Bruno & Marrone", "Pedro", "222.222.222-22", "5511982332556", "01/04/2023 22:40"),
    ("Oceano", "09/04/2023 22:00", "Djavan", "Suellen", "777.777.777-77", "5521991012434", "09/04/2023 23:06")
]

In [6]:
def insert_data(conn, table_name, data):
    try:
        sql = f"INSERT INTO {table_name} VALUES ({','.join(['?']*len(data))})"
        c = conn.cursor()
        c.execute(sql, data)
    except sqlite3.IntegrityError as e:
        if "UNIQUE constraint failed" in str(e):
            pass
        else:
            print(e)


for d in data:
    evento_inicio = datetime.strptime(d[1], "%d/%m/%Y %H:%M").strftime('%Y-%m-%d %H:%M:%S')
    insert_data(conn, "Dim_Evento", [d[0], evento_inicio, d[2]])
    
    insert_data(conn, "Dim_Cliente", [d[4], d[3], d[5]])
    
    data_hora = datetime.strptime(d[6], "%d/%m/%Y %H:%M").strftime('%Y-%m-%d %H:%M:%S')
    insert_data(conn, "Dim_Data_Hora", [data_hora, data_hora.split()[0], data_hora.split()[1]])
    
    insert_data(conn, "Fato_Acessos_Catraca", [d[0], d[4], data_hora])

conn.commit()

In [7]:
for table_name in table_names:
    print(f"Table: {table_name}")
    cursor.execute(f"SELECT * FROM {table_name}")
    for row in cursor.fetchall():
        print(row)
    print("\n")

Table: Dim_Evento
('Funn Festival', '2023-04-01 22:00:00', 'Bruno & Mamone')
('Fuzuê', '2023-04-02 22:00:00', 'Michel Teló')
('Comédia Stand Up', '2023-04-03 22:00:00', 'Fábio Rabin')
('2 Minutes To Midnight', '2023-04-05 22:00:00', 'Iron Maiden')
('Oceano', '2023-04-09 22:00:00', 'Djavan')


Table: Dim_Cliente
('111.111.111-11', 'Lucas', '5513992123212')
('222.222.222-22', 'Pedro', '5511982332556')
('333.333.333-33', 'Leonardo', '5513991857732')
('444.444.444-44', 'Vivian', '5521997889889')
('555.555.555-55', 'Anne', '5511988272121')
('666.666.666-66', 'Rodrigo', '5511945311288')
('777.777.777-77', 'Suellen', '5521991012434')


Table: Dim_Data_Hora
('2023-04-01 22:43:00', '2023-04-01', '22:43:00')
('2023-04-02 22:31:00', '2023-04-02', '22:31:00')
('2023-04-03 22:12:00', '2023-04-03', '22:12:00')
('2023-04-04 22:01:00', '2023-04-04', '22:01:00')
('2023-04-05 22:21:00', '2023-04-05', '22:21:00')
('2023-04-03 22:04:00', '2023-04-03', '22:04:00')
('2023-04-07 22:06:00', '2023-04-07', '22:

### Consultas

In [8]:
query_a = """
SELECT 
    e.evento_nome,
    COUNT(*) AS pessoas_atrasadas
FROM 
    Fato_Acessos_Catraca fac
JOIN 
    Dim_Evento e ON fac.evento_nome = e.evento_nome
JOIN 
    Dim_Data_Hora dh ON fac.cliente_entrada = dh.cliente_entrada
WHERE 
    dh.hora > e.evento_inicio
GROUP BY 
    e.evento_nome;
"""

print("Consulta A: Quantas pessoas chegaram após o horário de início de cada evento?")
cursor.execute(query_a)
results = cursor.fetchall()

pd.set_option('display.expand_frame_repr', False)
df = pd.DataFrame(results, columns=["evento_nome", "pessoas_atrasadas"])
print(df)

Consulta A: Quantas pessoas chegaram após o horário de início de cada evento?
             evento_nome  pessoas_atrasadas
0  2 Minutes To Midnight                  2
1       Comédia Stand Up                  2
2          Funn Festival                  2
3                  Fuzuê                  2
4                 Oceano                  1


In [9]:
query_b = """
SELECT 
    COUNT(DISTINCT evento_nome) AS eventos_no_sabado
FROM 
    Dim_Evento
WHERE 
    strftime('%w', evento_inicio) = '6';
"""

print("Consulta B: Quantos eventos aconteceram no dia de sábado?")
cursor.execute(query_b)
results = cursor.fetchall()

pd.set_option('display.expand_frame_repr', False)
df = pd.DataFrame(results, columns=["eventos_no_sabado"])
print(df)

Consulta B: Quantos eventos aconteceram no dia de sábado?
   eventos_no_sabado
0                  1


In [10]:
conn.close()